# NVIDIA Gave Me a \$15K Data Science Workstation - here's what I did with it

This notebook is an adaptation of my Towards Data Science Article availabe [here](https://towardsdatascience.com/nvidia-gave-me-a-15k-data-science-workstation-heres-what-i-did-with-it-70cfb069fc35).

## 1. Download and Process Data

Here, I walked through the example with one Pubmed file, although you could repeat this process for every file in the directory. I explicitely chose a document that has newer abstracts. Make sure your computer has enough processing power to handle this part of the process.

In [32]:
# download single Pubmed XML from the directory
!wget https://mbr.nlm.nih.gov/Download/Baselines/2019/pubmed19n0972.xml.gz
# unzip it
!gunzip pubmed19n0972.xml.gz

--2020-03-02 12:06:33--  https://mbr.nlm.nih.gov/Download/Baselines/2019/pubmed19n0972.xml.gz
Resolving mbr.nlm.nih.gov (mbr.nlm.nih.gov)... 130.14.53.15, 2607:f220:41e:7053::15
Connecting to mbr.nlm.nih.gov (mbr.nlm.nih.gov)|130.14.53.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8755134 (8.3M) [application/x-gzip]
Saving to: ‘pubmed19n0972.xml.gz’

100%[======================================>] 8,755,134   1.45MB/s   in 5.8s   

2020-03-02 12:06:39 (1.45 MB/s) - ‘pubmed19n0972.xml.gz’ saved [8755134/8755134]



Now we need to parse the XML to CSV.

In [35]:
from bs4 import BeautifulSoup

RAW_DATA = "pubmed19n0972.xml"

def parse_pubmed_xml(path, field):
    with open(path, "r") as f:
        text = f.read()

    soup = BeautifulSoup(text,"xml")
    documents = [x.text for x in soup.find_all(field)]
    
    return documents

In [1]:
# soup.find_all("PubmedArticle")[1]

In [7]:
a = soup.find_all("Abstract")

In [22]:
a[8]

<Abstract>
<AbstractText>A serum agglutinin reactive with red cells in the presence of polycarboxyl groups is reported. It is likely that this represents an additional example of the type of agglutinin previously described as agglutinating red cells in the absence of ionized calcium. Experimental evidence is presented indicating that it is free polycarboxyl groups that potentiate agglutination and that any metal ion, such as calcium, capable of chelating with these groups will prove to be inhibitory.</AbstractText>
</Abstract>

In [ ]:
text[0:1000000]

In [ ]:
import sys
sys.path = ['', '/data/python/anaconda3/envs/rapids-env/lib/python37.zip', '/data/python/anaconda3/envs/rapids-env/lib/python3.7', '/data/python/anaconda3/envs/kyle-test/lib/python3.7/lib-dynload', '/home/gallak12/.local/lib/python3.7/site-packages', '/data/python/anaconda3/envs/kyle-test/lib/python3.7/site-packages']

In [ ]:
import sys 
sys.path

In [ ]:
!ls ~/.local/share/jupyter/

In [ ]:
!which python

In [ ]:
import cuml

In [ ]:
import sys
sys.path.append("/data/python/anaconda3/bin")